VRC SSE　SI　确定簇数不一致时以SI 为准
SI 多数时为2，可以不去看2的值

密度聚类里 聚类结果为-1 的表示孤立点

In [77]:
import pandas as pd
import jieba
import numpy as np

In [78]:
import os
os.getcwd()

'C:\\Users\\acer\\Desktop\\125\\python\\文本'

In [81]:
d = pd.read_csv("社会新闻.csv",engine="python")
d = d.loc[:,["ext5","keywords","title"]]


In [89]:
d.head()

,ext5,keywords
0,原标题：孩子打开冰箱，脸上缝38针！夏天到了，这些东西放进冰箱要慎重\r\n炎炎夏日，温度逐...,"爆炸,冰箱,啤酒瓶"
1,原标题：中科院研究生被高中同学连刺数刀身亡，却仅仅因为……\r\n来源：中国之声中科院研究生...,"中科院,大学生,林森浩"
2,中科院研究生被高中同学连刺数刀身亡，却仅仅因为……\r\n原创：中国之声中国之声今天中科院研...,"中科院,大学生"
3,原标题：那个亲手掐死两个女儿的父亲\r\n来源：中国新闻周刊男子自述杀两亲生女过程:同时掐两...,"嫌疑人,肖珍,韦越"
4,原标题：农业农村部：猪病毒“塞内卡”传入我国，各地应及时诊断报告\r\n农业农村部办公厅关于...,"疫情,塞内卡,流行病"


In [88]:
d.ext5 = np.where(d.ext5.isna(),d.title,np.where(d.title.isna(),d.ext5,d.ext5+d.title))
d = d.iloc[:,:-1]

In [91]:
d.ext5 = d.ext5.apply(lambda x:jieba.lcut(x))

In [104]:
d.ext5 = d.ext5.apply(lambda x:pd.Series(x).str.replace("[\\d+ \\r\\n]","").values)  

In [107]:
d.ext5 = d.ext5.apply(lambda x: pd.Series(x)[pd.Series(x).str.len()>1].values)

In [141]:
d.ext5[0] 

array(['标题', '孩子', '打开', '冰箱', '脸上', '夏天', '这些', '东西', '放进', '冰箱', '慎重',
       '炎炎夏日', '温度', '逐渐', '上升', '今年', '夏天', '出新', '高度', '赶紧', '拿出', '一瓶',
       '可乐', '压惊', '孩子', '冰箱', '脸上', '这些', '东西', '夏天', '冰箱', '慎重'],
      dtype=object)

In [123]:
d.head()

,ext5,keywords,标签
0,"[标题, 孩子, 打开, 冰箱, 脸上, 夏天, 这些, 东西, 放进, 冰箱, 慎重, 炎...","[爆炸, 冰箱, 啤酒瓶]","[冰箱, 夏天, 脸上]"
1,"[标题, 中科院, 研究生, 高中同学, 连刺, 数刀, 身亡, 仅仅, 因为, 来源, 中...","[中科院, 大学生, 林森浩]","[连刺, 研究生, 数刀]"
2,"[中科院, 研究生, 高中同学, 连刺, 数刀, 身亡, 仅仅, 因为, 原创, 中国, 之...","[中科院, 大学生]","[身亡, 高中同学, 之声]"
3,"[标题, 那个, 亲手, 掐死, 两个, 女儿, 父亲, 来源, 中国, 新闻周刊, 男子,...","[嫌疑人, 肖珍, 韦越]","[来源, 颈部, 两人]"
4,"[标题, 农业, 农村部, 病毒, 塞内, 传入, 我国, 各地, 及时, 诊断, 报告, ...","[疫情, 塞内卡, 流行病]","[病毒, 农村部, 塞内]"


In [119]:
d["标签"]=d.ext5.apply(lambda x: list(pd.Series(x).value_counts()[:3].index))

In [122]:
d.keywords = d.keywords.str.split(",")

In [131]:
d= d.loc[~d.keywords.isna(),:]

In [135]:
d["dui"] = d.apply(lambda x: pd.Series(x[2]).isin(x[1]).sum()>0, axis=1)

In [137]:
d.head(10)

,ext5,keywords,标签,dui
0,"[标题, 孩子, 打开, 冰箱, 脸上, 夏天, 这些, 东西, 放进, 冰箱, 慎重, 炎...","[爆炸, 冰箱, 啤酒瓶]","[冰箱, 夏天, 脸上]",True
1,"[标题, 中科院, 研究生, 高中同学, 连刺, 数刀, 身亡, 仅仅, 因为, 来源, 中...","[中科院, 大学生, 林森浩]","[连刺, 研究生, 数刀]",False
2,"[中科院, 研究生, 高中同学, 连刺, 数刀, 身亡, 仅仅, 因为, 原创, 中国, 之...","[中科院, 大学生]","[身亡, 高中同学, 之声]",False
3,"[标题, 那个, 亲手, 掐死, 两个, 女儿, 父亲, 来源, 中国, 新闻周刊, 男子,...","[嫌疑人, 肖珍, 韦越]","[来源, 颈部, 两人]",False
4,"[标题, 农业, 农村部, 病毒, 塞内, 传入, 我国, 各地, 及时, 诊断, 报告, ...","[疫情, 塞内卡, 流行病]","[病毒, 农村部, 塞内]",False
5,"[标题, 仪陇, 通报, 交警, 中队长, 警棍, 争执, 不当, 行为, 已约, 针对, ...","[仪陇, 警棍, 中队长]","[交警, 警棍, 争执]",True
6,"[标题, 城管, 殴打, 小贩, 一个, 惩治, 粗暴, 执法, 样本, 曾几何时, 城管,...","[城管, 殴打, 小贩, 执法]","[城管, 样本, 小贩]",True
7,"[标题, 乖乖女, 男友, 父母, 支持, 反对, 只要, 默默, 盯梢, 恋爱, 父母, ...","[乖乖女, 父母, 谈恋爱]","[父母, 男友, 乖乖女]",True
8,"[标题, 老师, 扔书, 左眼, 致盲, 西安, 一中, 学生, 获赔, 万元, 西安, 晚...","[校方, 左眼, 中学生]","[左眼, 老师, 小张]",True
9,"[标题, 墨西哥, 球迷, 世界杯赛, 场上, 辱骂, 德国, 门将, 同性恋, 该国, 足...","[世界杯, 墨西哥, 国际足联]","[墨西哥, 球迷, 德国]",True


In [138]:
d.dui.sum()

5777

In [139]:
d.shape

(8726, 4)

In [140]:
5777/8726

0.6620444648177859

In [34]:
d.ext5 = d.ext5.apply(lambda x:jieba.lcut(x))
d.ext5 = d.ext5.apply(lambda x:pd.Series(x).str.replace("[\\d+ \\r\\n]","").values)
d.ext5 = d.ext5.apply(lambda x:pd.Series(x)[pd.Series(x).str.len()>1].values)

In [1]:
d["标签"]=d.ext5.apply(lambda x: list(pd.Series(x).value_counts()[:3].to_frame().index))

NameError: name 'd' is not defined

In [53]:
d.keywords=d.keywords.str.split(",")

In [69]:
d = d.loc[~d.keywords.isna(),:]

In [75]:
d["dui"]=d.apply(lambda x:pd.Series(x[2]).isin(x[1]).sum()>0,axis=1)

c:\users\acer\pycharmprojects\base\venv1\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [76]:
d.dui.sum()/len(d)

0.6620444648177859